# Chapter 18: Advanced Routing Patterns

The Next.js App Router introduces sophisticated routing capabilities that go beyond traditional page-based navigation. Parallel Routes allow you to render multiple pages simultaneously in the same layout, Intercepting Routes enable modal-like experiences while preserving URL state, and advanced grouping strategies help you organize complex applications without cluttering your URL structure.

By the end of this chapter, you'll master Parallel Routes for complex dashboard layouts, Intercepting Routes for seamless modal experiences, advanced Route Group patterns for organizational flexibility, Multi-Zone architectures for micro-frontend deployments, and sophisticated internationalization routing strategies.

## 18.1 Parallel Routes

Parallel Routes allow you to simultaneously render multiple pages in the same layout, independently navigable via separate URL segments.

### Basic Parallel Route Structure

Use the `@folder` convention to create named slots that render simultaneously:

```typescript
// File structure
app/
├── layout.tsx
├── @dashboard/           // Parallel route slot
│   ├── page.tsx         // Default content
│   └── settings/
│       └── page.tsx
├── @sidebar/            // Another parallel slot
│   ├── page.tsx
│   └── loading.tsx
├── page.tsx             // Main content
└── default.tsx          // Fallback when slot not matched

// app/layout.tsx
export default function Layout({
  children,
  dashboard,
  sidebar,
}: {
  children: React.ReactNode;
  dashboard: React.ReactNode;
  sidebar: React.ReactNode;
}) {
  return (
    <div className="flex h-screen">
      <aside className="w-64 bg-gray-100">
        {sidebar}
      </aside>
      <main className="flex-1 p-6">
        {dashboard}
      </main>
      <div className="w-80 border-l">
        {children}
      </div>
    </div>
  );
}
```

### Complex Dashboard Layout

Build sophisticated admin dashboards with independent navigation:

```typescript
// app/complex-layout/layout.tsx
export default function ComplexLayout({
  children,
  team,
  analytics,
  notifications,
}: {
  children: React.ReactNode;
  team: React.ReactNode;
  analytics: React.ReactNode;
  notifications: React.ReactNode;
}) {
  return (
    <div className="min-h-screen bg-gray-50">
      <header className="bg-white border-b p-4">
        <h1 className="text-xl font-bold">Admin Dashboard</h1>
      </header>
      
      <div className="grid grid-cols-12 gap-4 p-4">
        {/* Left sidebar - Team members */}
        <div className="col-span-3">
          {team}
        </div>
        
        {/* Main content area */}
        <div className="col-span-6">
          {children}
        </div>
        
        {/* Right sidebar - Split into two sections */}
        <div className="col-span-3 space-y-4">
          <div className="bg-white rounded-lg shadow p-4">
            <h2 className="font-semibold mb-2">Analytics</h2>
            {analytics}
          </div>
          
          <div className="bg-white rounded-lg shadow p-4">
            <h2 className="font-semibold mb-2">Notifications</h2>
            {notifications}
          </div>
        </div>
      </div>
    </div>
  );
}

// app/complex-layout/@team/page.tsx
async function getTeamMembers() {
  return db.user.findMany({ take: 10 });
}

export default async function TeamSlot() {
  const members = await getTeamMembers();
  
  return (
    <div className="bg-white rounded-lg shadow">
      <ul className="divide-y">
        {members.map(member => (
          <li key={member.id} className="p-3 flex items-center gap-3">
            <img src={member.image} className="w-8 h-8 rounded-full" />
            <span>{member.name}</span>
          </li>
        ))}
      </ul>
    </div>
  );
}

// app/complex-layout/@analytics/page.tsx
export default async function AnalyticsSlot() {
  const stats = await getAnalytics();
  
  return (
    <div className="space-y-2">
      <div className="flex justify-between">
        <span>Views</span>
        <span className="font-bold">{stats.views}</span>
      </div>
      <div className="flex justify-between">
        <span>Users</span>
        <span className="font-bold">{stats.users}</span>
      </div>
    </div>
  );
}
```

### Handling unmatched routes with default.tsx

When a parallel route doesn't match the current URL, render a fallback:

```typescript
// app/@sidebar/default.tsx
export default function SidebarDefault() {
  return (
    <div className="p-4 text-gray-500">
      <p>Select an item to view details</p>
    </div>
  );
}

// app/@sidebar/[id]/page.tsx
export default async function SidebarItem({ params }: { params: { id: string } }) {
  const item = await getItem(params.id);
  
  return (
    <div className="p-4">
      <h2>{item.title}</h2>
      <p>{item.description}</p>
    </div>
  );
}
```

### Independent Loading States

Each parallel route can have its own loading and error boundaries:

```typescript
// app/@analytics/loading.tsx
export default function AnalyticsLoading() {
  return (
    <div className="animate-pulse space-y-2">
      <div className="h-4 bg-gray-200 rounded w-3/4"></div>
      <div className="h-4 bg-gray-200 rounded w-1/2"></div>
    </div>
  );
}

// app/@analytics/error.tsx
'use client';

export default function AnalyticsError({ error, reset }: { 
  error: Error; 
  reset: () => void 
}) {
  return (
    <div className="p-4 bg-red-50 text-red-700 rounded">
      <p>Failed to load analytics</p>
      <button onClick={reset} className="text-sm underline">
        Try again
      </button>
    </div>
  );
}
```

## 18.2 Intercepting Routes

Intercepting Routes allow you to intercept a route from another part of your application and render it within the current layout—perfect for modals and overlays.

### Intercepting Route Patterns

Use `(.)`, `(..)`, `(...)`, or `(..)(..)` to match different segment levels:

```typescript
// File structure
app/
├── feed/
│   ├── page.tsx              // Main feed page
│   └── @modal/               // Intercepting route slot
│       ├── (.)photo/         // Intercept /photo from /feed
│       │   └── [id]/
│       │       └── page.tsx  // Modal version of photo
│       └── default.tsx       // No modal when not intercepting
├── photo/
│   └── [id]/
│       └── page.tsx          // Full photo page (when accessed directly)
└── layout.tsx

// (.)        → Intercept same level segment
// (..)       → Intercept one level up
// (..)(..)   → Intercept two levels up
// (...)      → Intercept root level
```

### Photo Feed Modal Example

The classic intercepting routes use case: clicking a photo opens a modal, but refreshing shows the full page:

```typescript
// app/feed/layout.tsx
export default function FeedLayout({
  children,
  modal,
}: {
  children: React.ReactNode;
  modal: React.ReactNode;
}) {
  return (
    <>
      {children}
      {modal}
    </>
  );
}

// app/feed/@modal/default.tsx
export default function Default() {
  return null; // Render nothing when no modal
}

// app/feed/@modal/(.)photo/[id]/page.tsx
'use client';

import { useRouter } from 'next/navigation';
import { Dialog, DialogContent } from '@/components/ui/dialog';

export default function PhotoModal({ params }: { params: { id: string } }) {
  const router = useRouter();
  
  return (
    <Dialog open onOpenChange={() => router.back()}>
      <DialogContent className="max-w-4xl">
        <PhotoDetail id={params.id} />
      </DialogContent>
    </Dialog>
  );
}

// app/photo/[id]/page.tsx
export default async function PhotoPage({ params }: { params: { id: string } }) {
  const photo = await getPhoto(params.id);
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-2xl font-bold mb-4">{photo.title}</h1>
      <img src={photo.url} className="w-full rounded-lg" />
      <p className="mt-4">{photo.description}</p>
    </div>
  );
}
```

### Complex Interception Patterns

Intercept routes from different levels for shopping cart sidebars:

```typescript
// app/shop/layout.tsx
export default function ShopLayout({
  children,
  cart,
}: {
  children: React.ReactNode;
  cart: React.ReactNode;
}) {
  return (
    <div className="flex">
      <main className="flex-1">{children}</main>
      {cart && <aside className="w-96 border-l">{cart}</aside>}
    </div>
  );
}

// app/shop/@cart/(..)cart/page.tsx
// Intercepts /cart when accessed from /shop/*
'use client';

import { Sheet, SheetContent } from '@/components/ui/sheet';
import { useRouter } from 'next/navigation';

export default function CartSidebar() {
  const router = useRouter();
  
  return (
    <Sheet open onOpenChange={() => router.back()}>
      <SheetContent className="w-96">
        <CartContents />
      </SheetContent>
    </Sheet>
  );
}

// app/cart/page.tsx
// Full cart page when accessed directly
export default function CartPage() {
  return (
    <div className="container mx-auto py-8 max-w-2xl">
      <h1 className="text-3xl font-bold mb-8">Your Cart</h1>
      <CartContents />
      <CheckoutButton />
    </div>
  );
}
```

## 18.3 Route Groups Advanced Usage

Route Groups (folders with parentheses) allow you to organize routes without affecting the URL structure.

### Layout Organization

Group routes that share layouts without adding segments to the URL:

```typescript
// File structure
app/
├── (marketing)/              // Group: No URL segment
│   ├── layout.tsx           // Marketing layout (different nav, footer)
│   ├── page.tsx             // → /
│   ├── about/
│   │   └── page.tsx         // → /about
│   └── pricing/
│       └── page.tsx         // → /pricing
├── (shop)/                  // Group: Shopping experience
│   ├── layout.tsx           // Shop layout with cart, categories
│   ├── shop/
│   │   └── page.tsx         // → /shop
│   └── products/
│       └── [id]/
│           └── page.tsx     // → /products/123
└── (dashboard)/             // Group: App dashboard
    ├── layout.tsx           // Dashboard layout with sidebar
    ├── dashboard/
    │   └── page.tsx         // → /dashboard
    └── settings/
        └── page.tsx         // → /settings
```

### Multiple Root Layouts

Create completely different layouts for different sections:

```typescript
// app/(marketing)/layout.tsx
export default function MarketingLayout({ children }: { children: React.ReactNode }) {
  return (
    <div className="marketing-theme">
      <nav className="sticky top-0 bg-white/80 backdrop-blur">
        {/* Marketing nav with CTA buttons */}
      </nav>
      {children}
      <footer className="bg-gray-900 text-white">
        {/* Marketing footer */}
      </footer>
    </div>
  );
}

// app/(dashboard)/layout.tsx
export default function DashboardLayout({ children }: { children: React.ReactNode }) {
  return (
    <div className="dashboard-theme flex h-screen">
      <Sidebar />
      <main className="flex-1 overflow-auto bg-gray-50">
        {children}
      </main>
    </div>
  );
}
```

### Route Group Organization for Large Apps

Use groups to organize by feature rather than by page type:

```typescript
// app/
├── (auth)/                  // Authentication flows
│   ├── login/
│   ├── register/
│   └── forgot-password/
├── (blog)/                  // Content management
│   ├── blog/
│   ├── blog/[slug]/
│   └── author/[name]/
└── (api-docs)/              // API documentation
    ├── api-reference/
    └── webhooks/
```

## 18.4 Modal and Overlay Patterns

Combine Parallel and Intercepting Routes for sophisticated modal experiences.

### Photo Gallery with Modal Detail

Full implementation of the modal pattern with state preservation:

```typescript
// app/gallery/@modal/(.)photo/[id]/page.tsx
'use client';

import { useCallback, useEffect } from 'react';
import { useRouter } from 'next/navigation';
import Image from 'next/image';

export default function PhotoModal({ 
  params 
}: { 
  params: { id: string } 
}) {
  const router = useRouter();
  
  // Close on escape key
  useEffect(() => {
    const handleKeyDown = (e: KeyboardEvent) => {
      if (e.key === 'Escape') router.back();
    };
    window.addEventListener('keydown', handleKeyDown);
    return () => window.removeEventListener('keydown', handleKeyDown);
  }, [router]);
  
  const onDismiss = useCallback(() => {
    router.back();
  }, [router]);
  
  return (
    <div className="fixed inset-0 z-50 bg-black/80 flex items-center justify-center p-4">
      <div 
        className="absolute inset-0" 
        onClick={onDismiss}
      />
      <div className="relative bg-white rounded-lg overflow-hidden max-w-4xl w-full max-h-[90vh]">
        <button 
          onClick={onDismiss}
          className="absolute top-4 right-4 z-10 bg-black/50 text-white rounded-full p-2"
        >
          ✕
        </button>
        <PhotoContent id={params.id} />
      </div>
    </div>
  );
}

// app/gallery/@modal/default.tsx
export default function Default() {
  return null;
}
```

### Nested Modal Routing

Handle stacked modals with route interception:

```typescript
// app/feed/@modal/(.)item/[id]/(.)comments/page.tsx
// Opens comments modal on top of item modal
'use client';

import { useRouter } from 'next/navigation';

export default function CommentsModal({ params }: { params: { id: string } }) {
  const router = useRouter();
  
  return (
    <div className="fixed inset-0 z-[60] bg-black/50 flex items-end sm:items-center justify-center">
      <div className="bg-white w-full max-w-lg rounded-t-lg sm:rounded-lg p-6">
        <h2 className="text-xl font-bold mb-4">Comments</h2>
        <CommentsList itemId={params.id} />
        <button 
          onClick={() => router.back()}
          className="mt-4 w-full py-2 border rounded"
        >
          Close Comments
        </button>
      </div>
    </div>
  );
}
```

### Drawer/Sheet Patterns

Use intercepting routes for slide-out panels:

```typescript
// app/inbox/@details/(.)mail/[id]/page.tsx
'use client';

import { useRouter } from 'next/navigation';
import { Sheet, SheetContent, SheetHeader, SheetTitle } from '@/components/ui/sheet';

export default function MailDrawer({ params }: { params: { id: string } }) {
  const router = useRouter();
  
  return (
    <Sheet open onOpenChange={() => router.back()}>
      <SheetContent className="w-[600px] sm:max-w-[600px]">
        <SheetHeader>
          <SheetTitle>Email Details</SheetTitle>
        </SheetHeader>
        <MailContent id={params.id} />
      </SheetContent>
    </Sheet>
  );
}
```

## 18.5 Multi-Zone Applications

Multi-Zone applications allow you to deploy multiple Next.js apps under the same domain, enabling micro-frontend architectures.

### Zone Configuration

Set up multiple Next.js apps as zones:

```javascript
// next.config.js for main app
module.exports = {
  async rewrites() {
    return [
      {
        source: '/shop',
        destination: 'https://shop.yoursite.com/shop',
      },
      {
        source: '/shop/:path*',
        destination: 'https://shop.yoursite.com/shop/:path*',
      },
      {
        source: '/blog',
        destination: 'https://blog.yoursite.com/blog',
      },
      {
        source: '/blog/:path*',
        destination: 'https://blog.yoursite.com/blog/:path*',
      },
    ];
  },
};
```

### Shared Components Across Zones

Maintain consistency with shared UI packages:

```typescript
// packages/ui/components/Button.tsx
// Shared component used across all zones

// apps/main/app/page.tsx
import { Button } from '@company/ui';

// apps/shop/app/page.tsx  
import { Button } from '@company/ui';

// Both apps use the same component source
```

### Authentication Across Zones

Share auth state using cookies and middleware:

```typescript
// middleware.ts in each zone
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Check for auth cookie set by main app
  const token = request.cookies.get('auth-token');
  
  if (!token && request.nextUrl.pathname.startsWith('/shop/account')) {
    // Redirect to main app's login, but remember where to return
    const returnUrl = encodeURIComponent(request.url);
    return NextResponse.redirect(
      new URL(`/login?returnTo=${returnUrl}`, request.url)
    );
  }
  
  return NextResponse.next();
}
```

## 18.6 Custom Route Configurations

Configure advanced routing behaviors in `next.config.js`.

### Rewrites and Redirects

Handle legacy URLs and route masking:

```javascript
// next.config.js
module.exports = {
  async redirects() {
    return [
      {
        source: '/old-portfolio/:path*',
        destination: '/work/:path*',
        permanent: true, // 301 redirect
      },
      {
        source: '/admin',
        destination: '/dashboard',
        permanent: false, // 307 temporary
      },
      // Redirect based on cookie
      {
        source: '/',
        has: [
          {
            type: 'cookie',
            key: 'visited',
            value: 'true',
          },
        ],
        permanent: false,
        destination: '/welcome-back',
      },
    ];
  },
  
  async rewrites() {
    return {
      beforeFiles: [
        // Apply before checking files/pages
        {
          source: '/api/legacy/:path*',
          destination: 'https://legacy-api.example.com/:path*',
        },
      ],
      afterFiles: [
        // Apply after checking pages, before 404
        {
          source: '/product/:id',
          destination: '/item/:id',
        },
      ],
      fallback: [
        // Only apply if no page matches
        {
          source: '/:path*',
          destination: '/catch-all/:path*',
        },
      ],
    };
  },
};
```

### Headers and Security

Add custom headers per route:

```javascript
// next.config.js
module.exports = {
  async headers() {
    return [
      {
        source: '/api/:path*',
        headers: [
          { key: 'Access-Control-Allow-Credentials', value: 'true' },
          { key: 'Access-Control-Allow-Origin', value: 'https://app.yoursite.com' },
          { key: 'Access-Control-Allow-Methods', value: 'GET,POST,PUT,DELETE,OPTIONS' },
        ],
      },
      {
        source: '/:path*',
        headers: [
          {
            key: 'X-DNS-Prefetch-Control',
            value: 'on',
          },
          {
            key: 'Strict-Transport-Security',
            value: 'max-age=63072000; includeSubDomains; preload',
          },
        ],
      },
    ];
  },
};
```

### Trailing Slashes and Clean URLs

Control URL formatting:

```javascript
// next.config.js
module.exports = {
  trailingSlash: true, // /about → /about/
  
  // Or handle dynamically
  skipTrailingSlashRedirect: true,
  
  // Clean URLs without .html
  cleanDistDir: true,
};
```

## 18.7 Internationalization (i18n)

Advanced routing for multi-language applications.

### Subpath Routing

Serve different languages under path prefixes:

```javascript
// next.config.js
module.exports = {
  i18n: {
    locales: ['en-US', 'fr-FR', 'es-ES', 'de-DE'],
    defaultLocale: 'en-US',
    localeDetection: true, // Auto-detect from browser
    
    // Domains for each locale (optional)
    domains: [
      {
        domain: 'example.com',
        defaultLocale: 'en-US',
      },
      {
        domain: 'example.fr',
        defaultLocale: 'fr-FR',
      },
    ],
  },
};
```

### Locale-Aware Navigation

Handle locale in links and routing:

```typescript
// components/locale-switcher.tsx
'use client';

import { usePathname, useRouter } from 'next/navigation';

export function LocaleSwitcher({ currentLocale }: { currentLocale: string }) {
  const router = useRouter();
  const pathname = usePathname();
  
  const switchLocale = (locale: string) => {
    // Replace locale in path
    const newPath = pathname.replace(`/${currentLocale}`, `/${locale}`);
    router.push(newPath);
  };
  
  return (
    <select 
      value={currentLocale} 
      onChange={(e) => switchLocale(e.target.value)}
    >
      <option value="en-US">English</option>
      <option value="fr-FR">Français</option>
      <option value="es-ES">Español</option>
    </select>
  );
}

// app/[locale]/layout.tsx
export default async function LocaleLayout({
  children,
  params: { locale },
}: {
  children: React.ReactNode;
  params: { locale: string };
}) {
  // Validate locale
  if (!locales.includes(locale)) {
    notFound();
  }
  
  // Load messages for locale
  const messages = await import(`@/messages/${locale}.json`);
  
  return (
    <NextIntlClientProvider locale={locale} messages={messages.default}>
      {children}
    </NextIntlClientProvider>
  );
}
```

### RTL Language Support

Handle right-to-left layouts:

```typescript
// app/[locale]/layout.tsx
import { dir } from 'i18next';

const locales = ['en', 'ar', 'he'];

export default function RootLayout({
  children,
  params: { locale },
}: {
  children: React.ReactNode;
  params: { locale: string };
}) {
  return (
    <html lang={locale} dir={dir(locale)}>
      <body className={locale === 'ar' || locale === 'he' ? 'rtl' : 'ltr'}>
        {children}
      </body>
    </html>
  );
}

// tailwind.config.js
module.exports = {
  theme: {
    extend: {
      // Tailwind CSS Logical Properties for RTL support
      spacing: {
        'safe-start': 'env(safe-area-inset-left)',
        'safe-end': 'env(safe-area-inset-right)',
      },
    },
  },
};
```

### Locale-Specific Route Groups

Different layouts for different regions:

```typescript
// app/
├── [locale]/
│   ├── (marketing)/
│   │   ├── layout.tsx      // Marketing layout
│   │   └── page.tsx        // Localized homepage
│   └── (platform)/
│       ├── layout.tsx      // App layout
│       └── dashboard/
└── api/                    // Non-localized API routes
```

## Key Takeaways from Chapter 18

1. **Parallel Routes**: Use the `@folder` convention to render multiple pages simultaneously in the same layout. Create complex dashboard layouts with independent navigation slots (sidebar, main content, analytics panels) that can each have their own loading and error states. Always provide a `default.tsx` for unmatched slots.

2. **Intercepting Routes**: Implement modal-like experiences using `(.)` for same-level interception, `(..)` for parent level, and `(...)` for root interception. This allows users to view item details in a modal while maintaining context, with the URL reflecting the full route for shareability and direct access.

3. **Advanced Route Groups**: Use parentheses `(group)` to organize routes by feature or layout type without affecting URL structure. Create multiple root layouts for completely different experiences (marketing vs. dashboard) while keeping URLs clean (no `/marketing/about`, just `/about`).

4. **Modal and Overlay Patterns**: Combine Parallel and Intercepting Routes to create sophisticated modal systems where clicking an item opens an overlay, but refreshing the page shows the full standalone version. Handle stacked modals and drawer patterns while preserving browser history and back button functionality.

5. **Multi-Zone Applications**: Deploy multiple Next.js applications under the same domain using rewrites in `next.config.js`. Share authentication state via cookies across zones, maintain consistent UI through shared component packages, and enable independent deployment of micro-frontends.

6. **Custom Route Configurations**: Use `next.config.js` to handle legacy URL redirects, implement path rewrites for API proxying or clean URLs, add security headers per route, and configure locale-specific domains or subpaths for internationalization.

7. **Internationalization Routing**: Implement subpath-based i18n (`/en`, `/fr`) or domain-based routing with automatic locale detection. Handle RTL languages by setting the `dir` attribute dynamically, and organize localized content using route groups to maintain clean code structure while supporting multiple languages.

## Coming Up Next

**Chapter 19: Error Handling & Debugging**

Now that you've mastered advanced routing patterns, it's crucial to ensure your application handles failures gracefully. In Chapter 19, we'll explore Error Boundaries for catching React errors, global error handling strategies, the `notFound()` function for missing data, custom error pages, debugging techniques for Server Components, logging and monitoring strategies, and common error patterns in Next.js applications. You'll learn how to build resilient applications that fail gracefully and provide excellent developer experiences.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='17. database_integration.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. Production_ready_development/19. error_handling_and_debugging.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
